# Modelo de recomendacion de peliculas

In [ ]:
# modulos a utilizar
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt
import pandas as pd, glob,pickle

Cargar datos

In [ ]:
data_= pd.concat(
    [pd.read_csv(f) for f in glob.glob('/content/ratings/*.csv')], 
    ignore_index=True)

Entrenar el modelo

In [ ]:
#se seleccionan columnas a utilizar
ratings_df = data_[['userId','rating','movieId']]
#eliminar valores nulos
ratings_df =ratings_df.dropna()
ratings_df['movieId'] = ratings_df['movieId'].astype(str)
#se escojen 5 millones de registros para hacer el entrenamiento
ratings_df=ratings_df[0:5000000]
#se remplaza las letras que identifican a las plataformas {a:1,d:2,h:3,n:4} ademas s se remplaza por el numero 9
ratings_df['movieId'] = ratings_df['movieId'].apply(
    lambda x: '19' + x[2:] if x[0]=='a' else '29' + x[2:] if x[0]=='d' else '39' + x[2:] if x[0]=='h' else '49' + x[2:] if x[0]=='n' else x)
#cambiar tipos de datos
ratings_df['movieId'] = ratings_df['movieId'].astype('float64')
ratings_df['userId'] = ratings_df['userId'].astype('float64').fillna(0)
ratings_df['rating'] = ratings_df['rating'].astype('float64').fillna(0)
ratings_df=ratings_df.fillna(0)

#dividir por columnas
userid = ratings_df['userId']
movieid = ratings_df['movieId']
score = ratings_df['rating'] 
# crear arreglos
userid = userid.to_numpy()
movieid = movieid.to_numpy()
score = score.to_numpy()
#crear arreglo bidimencional
ab = np.stack((userid, movieid), axis=1)

#red neuronal con 10 neuronas y 6 capas y dos parametros de entrada
capa1 = tf.keras.layers.Dense(units=10, input_shape=[2])
capa2 = tf.keras.layers.Dense(units=10)
capa3= tf.keras.layers.Dense(units=10)
capa4 = tf.keras.layers.Dense(units=10)
capa5 = tf.keras.layers.Dense(units=10)
capa6 = tf.keras.layers.Dense(units=10)
salida = tf.keras.layers.Dense(units=1)
#crear modelo Squential
modelo = tf.keras.Sequential([capa1,capa2,capa3,capa4,capa5,capa6,salida])
#compilar modelo
modelo.compile(optimizer=tf.keras.optimizers.Adam(0.002),
               loss='mean_squared_error')
#entrenamiento del modelo
historial = modelo.fit(ab,score,epochs=15,verbose=False)


Visualizar el progreso de la red

In [ ]:
plt.xlabel('epoca')
plt.ylabel('magnitud de perdida')
plt.plot(historial.history['loss'])

hacer una prediccion

In [ ]:
arreglo=np.array([120])
arr =np.array([493205])
predic = np.stack((arreglo,arr),axis=1)
resultado=modelo.predict([predic])
print(predic[0],resultado)

Exportar el modelo

In [ ]:
with open('modelo_recomendacion.pkl', 'wb') as archivo:
    pickle.dump(modelo, archivo)

In [ ]:
# cargar modelo
with open('modelo_recomendacion.pkl', 'rb') as archivo:
    modelo_cargado = pickle.load(archivo)
resultado =modelo_cargado.predict([predic])
print(predic[0],resultado)